<a href="https://colab.research.google.com/github/Adamsongy/adam_test/blob/main/naivebayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/tibaml1027/raw/main/poem_train.csv"
urlretrieve(url,"train.csv")
train_df = pd.read_csv("train.csv",encoding="utf-8")
url = "https://github.com/Elwing-Chou/tibaml1027/raw/main/poem_test.csv"
urlretrieve(url,"test.csv")
test_df = pd.read_csv("test.csv",encoding="utf_8")

In [ ]:
#train_df
test_df

In [3]:
# 示範如何作儲存
# dic = {
#     "店名":[],
#     "食記":[]
# }
# # ....
# dic["店名"].append("大腕")
# dic["食記"].append('真的很好吃, \n非常非常"好吃"')
# pd.DataFrame(dic).to_csv("eat.csv", encoding="utf-8", index=False)

In [ ]:
# 進到函式庫之前,所有東西換成數字
# 李白,杜甫,白居易->0,1,2
# series.replace({"李白":0, "杜甫":1})
# series.unique/value_counts
u=train_df["作者"].unique()

# name2idx={}
# for i,n in enumerate(u):
#   name2idx[n]=i
# 可試看看: [i ** 2 for i in range(5)]

name2idx = {n:i for i,n in enumerate(u)}
idx2name = {i:n for i,n in enumerate(u)}
y_train = train_df["作者"].replace(name2idx)
y_test = test_df["作者"].replace(name2idx)
y_test

In [ ]:
import jieba
def poemcut(p):
  return " ".join(jieba.cut(p))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [8]:
#vec.vocabuulary_

In [9]:
#print(x_train_couunt)
x_test_count

<30x52294 sparse matrix of type '<class 'numpy.int64'>'
	with 352 stored elements in Compressed Sparse Row format>

In [ ]:
# check標點和換行沒被算進去(。, \n, \r\n): KeyError才是對的
# vec.vocabulary_["\r\n"]
# vec.vocabulary_["。"]

In [11]:
from sklearn.naive_bayes import MultinomialNB
clf= MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [12]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test,pre),
      index=["{}(正確)".format(n) for n in u],
      columns=["{}(預測)".format(n) for n in u])

In [ ]:
testp = input("來首詩:")
testpcount = vec.transform([poemcut(testp)])
proba = clf.predict_proba(testpcount)[0]
for n, p in zip(u, proba):
    print(n, "的機率是:", p)